In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import copy
import itertools as it

## Transportation problem
TODO:
- Handle `supply != demand` scenario

### Prepare data

In [229]:
costs = np.array(
    [[3,  5,  7],
    [12, 10, 9],
    [13, 3,  9]]
)
supply = np.array([20, 40, 90]) # podaż
demand = np.array([50, 70, 30])  #popyt 

SIZE = costs.shape[0]

transportation = np.zeros((SIZE, SIZE))

# List of tuples (x, y) which describes position allowed to read
path: list[tuple] = []

### Create path
Create `path` variable, `transportation` matrix and substract values from `demand`/`supply` lists

In [230]:
d, s = 0, 0
while(True):   
    if d == SIZE or s == SIZE:
        break
        
    if demand[d] >= supply[s]:
        demand[d] -= supply[s]
        transportation[d, s] = supply[s]
        
        path.append((d, s))
        s += 1
    
    else:
        supply[s] -= demand[d]
        transportation[d, s] = demand[d]
        
        path.append((d, s))
        d += 1

In [106]:
transportation

array([[10.,  0., 10.,  0.],
       [ 0., 28.,  2.,  0.],
       [ 0.,  0., 15., 50.]])

In [105]:
path

[(0, 0), (0, 2), (1, 1), (1, 2), (2, 3), (2, 2)]

### Make zeros on the path

In [220]:
# Custom path for testing
path = [(0, 0), (0, 1), (1, 2), (2, 1), (2, 2)]

In [231]:
O = np.zeros((SIZE,)) # alpha
D = np.zeros((SIZE,)) # beta
sO = np.full((SIZE,), False, dtype=bool) # state O
sO[0] = True
sD = np.full((SIZE,), False, dtype=bool) # state D

c_path = path[:] # copy of path for modification

c = 0 # counter
while(True):
    i, j = c_path[c]
    # If one of the values is True and remaining False
    if sO[i] ^ sD[j]:
        if sO[i]:
            D[j] = -1 * (O[i] + D[j] + costs[i, j])
            sD[j] = True
        else:
            O[i] = -1 * (O[i] + D[j] + costs[i, j])
            sO[i] = True
        c += 1
    
    # If both values are True
    elif not (sO[i] & sD[j]):
        c_path.append(c_path.pop(c))
    
    # If both values are False
    else:
        print("Error - both values sO[i] and sD[j] contain False value!")
        break

    if c > len(c_path) - 1:
        break

##### Results

In [238]:
O[:, np.newaxis]

array([[ 0.],
       [-5.],
       [-5.]])

In [233]:
D

array([-3., -5., -4.])

## Find minimal value of transportation matrix
After substracting alpha and beta

### Sum costs, O and D (alpha, beta)

In [244]:
r_costs = costs + D + O[:, np.newaxis]
r_costs

array([[ 0.,  0.,  3.],
       [ 4.,  0.,  0.],
       [ 5., -7.,  0.]])

In [265]:
# Store minimum value index of r_costs
min_value_i = np.unravel_index(np.argmin(r_costs, axis=None), r_costs.shape)

# Check if minimum value is negative, if not program should break
if r_costs[min_value_i] >= 0:
    min_value_i = None

In [270]:
r_costs[[2, 1], [2, 1]]

array([-7.,  0.])

### Find delta indices

In [332]:
assert min_value_i != None
indices = []
a, b = min_value_i

for c1, c2 in it.product(range(1, SIZE), repeat=2):
    indices = [
                (a, b),
                ((a + c1)%3, b),
                (a, (b + c2)%3),
                ((a + c1)%3, (b + c2)%3)
                ]
    
    e = tuple(zip(*indices))
    if all((i in path for i in indices[1:])):
        break

##### Results

In [333]:
indices

[(2, 1), (1, 1), (2, 2), (1, 2)]

In [336]:
r_costs[tuple(zip(*indices))]

array([-7.,  0., -7.,  0.])

In [347]:
# Get the minimal value for minus delta (...[e][1::2])
e = tuple(zip(*indices))
min_transp = np.min(transportation[e][1::2])
min_transp

10.0

### Others

In [107]:
v = list(zip(*path))
costs[v[0], v[1]]

array([ 3,  5, 10,  9,  9])

In [283]:
list(it.product([1, 2], repeat=2))

[(1, 1), (1, 2), (2, 1), (2, 2)]

In [304]:
r_costs

array([[ 0.,  0.,  3.],
       [ 4.,  0.,  0.],
       [ 5., -7., -7.]])

In [ ]:
r_costs[]

In [300]:
r_costs[tuple(zip(*indices))]

array([-7.,  0.,  5.,  4.])

## The broker issue

In [173]:
# Unit transportation costs
costs = np.array([
        [8, 14, 17],
        [12, 9, 19]
    ]
)

# Popyt i podaż dla odbiorców O1, O2, ... o dostawców D1, D2, ...
supply = np.array([20, 30]) # podaż
demand = np.array([10, 28, 27])  #popyt 

# Handling unbalanced scenario (fictional supplier/receiver)
profit = np.array([])
# If the sum of demand and supply is neq add fictional supplier and receiver
if (demand_sum := demand.sum()) != (supply_sum := supply.sum()):
    supply = np.append(supply, [demand_sum])
    demand = np.append(demand, [supply_sum])
    profit = np.zeros((supply.size, demand.size))


# Koszty zakupów / Cena sprzedaży
order_costs = np.array([10, 12])
sell_costs = np.array([30, 25, 30])

SHAPE = costs.shape
# SIZE = costs.shape[0]

transportation = np.zeros((supply.size, demand.size))

# List of tuples (x, y) which describes position allowed to read
path: list[tuple] = []

### Create profit matrix

In [174]:
# Zysk jednostkowy
if profit.size == 0:
    profit = np.zeros(SHAPE)

for i, j in it.product(range(SHAPE[0]), range(SHAPE[1])):
    profit[i, j] = sell_costs[j] - order_costs[i] - costs[i, j]

In [9]:
profit

array([[12.,  1.,  3.],
       [ 6.,  4., -1.]])

### Create transportation costs matrix

In [175]:
d, s = 0, 0
# Create array of indices that are pointing array elements
# in a sorted order
profit_sorted = np.argsort(np.absolute(profit), axis=1).tolist()
while(True):
    if supply[s] != 0:
        d = profit_sorted[s].pop() 
        if demand[d] == 0:
            continue
        path.append((s, d))
        
        if demand[d] >= supply[s]:
            demand[d] -= supply[s]
            transportation[s, d] = supply[s]
            supply[s] = 0
            
        else:
            supply[s] -= demand[d]
            transportation[s, d] = demand[d]
            demand[d] = 0

    else:
        s += 1

    if s >= supply.size:
        break

In [118]:
transportation

array([[10.,  0., 10.,  0.],
       [ 0., 28.,  2.,  0.],
       [ 0.,  0., 15., 50.]])

In [119]:
profit

array([[12.,  1.,  3.,  0.],
       [ 6.,  4., -1.,  0.],
       [ 0.,  0.,  0.,  0.]])

### Find alpha and beta

In [176]:
# Declare alpha and beta
O = np.zeros((profit.shape[0],)) # alpha
D = np.zeros((profit.shape[1],)) # beta

# Helper variables
sO = np.full((profit.shape[0],), False, dtype=bool) # state O
sO[0] = True
sD = np.full((profit.shape[1],), False, dtype=bool) # state D

# Copy of path array (path should be read-only)
c_path = path[:] # copy of path for modification

c = 0 # counter
while(True):
    i, j = c_path[c]
    # If values are opposite (False, True or True, False)
    if sO[i] ^ sD[j]:
        if sO[i]:
            D[j] = 1 * (- O[i] - D[j] + profit[i, j])
            sD[j] = True
        else:
            O[i] = 1 * (- O[i] - D[j] + profit[i, j])
            sO[i] = True
        c += 1
    
    # If both values are True
    elif not (sO[i] & sD[j]):
        c_path.append(c_path.pop(c))
    
    # If both values are False
    else:
        print("Error - both values sO[i] and sD[j] contain False value!")
        break

    if c > len(c_path) - 1:
        break

In [121]:
O

array([ 0., -4., -3.])

In [122]:
D

array([12.,  8.,  3.,  3.])

### Find minimal value of transportation matrix
After substracting alpha and beta

#### Sum costs, O and D (alpha, beta)

In [177]:
r_profit = profit - D - O[:, np.newaxis]
r_profit

array([[ 0., -7.,  0., -3.],
       [-2.,  0.,  0.,  1.],
       [-9., -5.,  0.,  0.]])

In [178]:
# Store minimum value index of r_costs
max_value_i = np.unravel_index(np.argmax(r_profit, axis=None), r_profit.shape)

# Check if maximum value is positive, if not program should break
if r_profit[max_value_i] <= 0:
    max_value_i = None

#### Find delta indices

In [180]:
assert max_value_i != None
indices = []
a, b = max_value_i
c1_max, c2_max = r_profit.shape[0], r_profit.shape[1]

for c1, c2 in it.product(range(1, c1_max+1), range(1, c2_max+1)):
    indices = [
                (a, b),
                ((a + c1)%(c1_max), b),
                (a, (b + c2)%(c2_max)),
                ((a + c1)%(c1_max), (b + c2)%(c2_max))
                ]
    
    e = tuple(zip(*indices))
    if all((i in path for i in indices[1:])):
        break

In [181]:
indices

[(1, 3), (2, 3), (1, 2), (2, 2)]

In [182]:
r_profit[tuple(zip(*indices))]

array([1., 0., 0., 0.])

In [183]:
profit[tuple(zip(*indices))]

array([ 0.,  0., -1.,  0.])

### Find minimal value for minus delta

In [190]:
# Get the minimal value for minus delta (...[e][[1, 2]])
e = tuple(zip(*indices))
min_transp = np.min(transportation[e][[1, 2]])
min_transp

2.0

In [184]:
r_profit

array([[ 0., -7.,  0., -3.],
       [-2.,  0.,  0.,  1.],
       [-9., -5.,  0.,  0.]])

In [186]:
c1_max, c2_max

(3, 4)

### Subtract min delta

In [194]:
transportation

array([[10.,  0., 10.,  0.],
       [ 0., 28.,  0.,  2.],
       [ 0.,  0., 17., 48.]])

In [191]:
# Create 4-element array with delta and minus delta values
deltas = np.full((4,), fill_value=min_transp)
deltas[[1, 2]] = deltas[[1, 2]] * -1
deltas

In [193]:
# Add deltas to appropirate elements in the transportation array
transportation[tuple(zip(*indices))] += deltas

In [189]:
# Print transportation array
transportation[tuple(zip(*indices))]

array([ 0., 50.,  2., 15.])